In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

# 회로 생성
qc = QuantumCircuit(1, 1)
qc.h(0)          # Hadamard gate
qc.z(0)          # External stimulus: Z-gate (phase shift)
qc.measure(0, 0) # 측정

# 시뮬레이터 실행
sim = AerSimulator()
tqc = transpile(qc, sim)
job = sim.run(tqc, shots=1000)
result = job.result()
counts = result.get_counts()

# 시각화 및 저장
fig = plot_histogram(counts, title="QFM-CELL-A.4b — Z-gate inserted")
fig.savefig("result_A4b_1.png")
plt.close(fig)


In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.distance import cosine

# 정량 지표 함수 정의
def shannon_entropy(counts):
    total = sum(counts.values())
    probs = [v / total for v in counts.values()]
    return -sum(p * np.log2(p) for p in probs if p > 0)

def distribution_vector(counts, keys):
    total = sum(counts.values())
    return np.array([counts.get(k, 0) / total for k in keys])

# 회로 구성 함수
def build_circuit():
    qc = QuantumCircuit(1, 1)
    qc.h(0)
    qc.z(0)
    qc.measure(0, 0)
    return qc

# 시뮬레이터 설정
sim = AerSimulator()
shots = 1000
repetitions = 3
counts_list = []

# 실험 반복 실행
for i in range(repetitions):
    qc = build_circuit()
    tqc = transpile(qc, sim)
    job = sim.run(tqc, shots=shots)
    result = job.result()
    counts = result.get_counts()
    counts_list.append(counts)

    # 개별 결과 시각화
    fig = plot_histogram(counts, title=f"QFM-CELL-A.4b — Run {i+1}")
    fig.savefig(f"result_A4b_run{i+1}.png")
    plt.close(fig)

# 엔트로피 계산 및 시각화
entropies = [shannon_entropy(c) for c in counts_list]
plt.plot(range(1, repetitions + 1), entropies, marker='o')
plt.title("QFM-CELL-A.4b — Shannon Entropy Across Repeats")
plt.xlabel("Repeat")
plt.ylabel("Entropy")
plt.grid(True)
plt.savefig("entropy_plot.png")
plt.show()

# 유사도 로그 저장
with open("similarity_log.txt", "w", encoding="utf-8") as f:
    f.write("QFM-CELL-A.4b — Cosine Similarity Log\n-----------------------------\n")
    for i in range(repetitions - 1):
        c1, c2 = counts_list[i], counts_list[i + 1]
        keys = sorted(set(c1) | set(c2))
        v1 = distribution_vector(c1, keys)
        v2 = distribution_vector(c2, keys)
        sim = 1 - cosine(v1, v2)
        f.write(f"Run {i+1} vs Run {i+2}: {sim:.4f}\n")
